In [1]:
import sys
!{sys.executable} -m pip install 'pandas'

ERROR: Invalid requirement: "'pandas'"


In [2]:
import pandas as pd
from functools import reduce
import string

responses = pd.DataFrame()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
for i in range(15,21):
    event_df = pd.read_csv('responses/e{}_post.csv'.format(i))
    event_df['event'] = 'e{}'.format(i)
    responses = pd.concat([responses, event_df], ignore_index=True)

responses_cleaned = pd.DataFrame()
responses.shape

(83, 54)

Dropping all irrelevant columns in responses. Only the final columns will be added to responses_cleaned for analysis. Currently, there are 54 columns at the start of cleaning. The next step will drop 5 columns.

In [3]:
responses = responses.drop(columns=['#','Start Date (UTC)','Submit Date (UTC)','Network ID','Timestamp','Name','*Finally, reminder to please check out our Digital Skills Sprint application: bit.ly/ccdss2021 (happening this June to July 2021)*','Other'])

In [4]:
knowledge_cols = [colname for colname in responses.columns if colname.startswith('Select the option that best describes your knowledge')]
interest_cols = [colname for colname in responses.columns if colname.startswith('Select the option that best describes your interest')]
learn_cols = [colname for colname in responses.columns if colname.startswith('The webinar has taught me what I wanted to learn about')]
other_profession_cols = [colname for colname in responses.columns if colname.startswith('Have you considered other professions outside of')]

knowledge_dfs = [responses[colname] for colname in knowledge_cols]
interest_dfs = [responses[colname] for colname in interest_cols]
learn_dfs = [responses[colname] for colname in learn_cols]
other_profession_dfs = [responses[colname] for colname in other_profession_cols]

Rename columns to something more legible and combine similar questions:

In [5]:
responses_cleaned['event'] = responses['event'].astype(str)

email_cols = ["Firstly, what's your email address?",'Username','Email']
email_dfs = [responses[colname] for colname in email_cols]
responses_cleaned['email'] = reduce(lambda x,y: x.combine_first(y), email_dfs).astype(str).str.lower()

responses_cleaned['one_word'] = responses["One word to describe today's online experience"].astype(str).str.lower().str.replace('[{}]'.format(string.punctuation), '').str.split().str[0]
responses_cleaned['what_I_liked'] = responses["What I liked about today's session"].astype(str)
responses_cleaned['other_profession'] = reduce(lambda x,y: x.combine_first(y), other_profession_dfs).astype(str).str.lower()
responses_cleaned['e15_ama_feedback'] = responses["What is something you wished Monica & Ashton could've touched on?"].astype(str)
responses_cleaned['future_topics'] = responses["CareerContact is thinking of hosting more AMA webinars in the future. What topics would interest you?"].astype(str)

reason_cols = ["My school provides talks like these already, but I was just curious to hear what Monica & Ashton have to say","My school doesn't provide talks like these at all","I don't know who to ask about college applications overseas","My teacher told me to sign up for this","I heard that my friends were signing up, so I decided to sign up too"]
responses_cleaned['reason_curious'] = responses["My school provides talks like these already, but I was just curious to hear what Monica & Ashton have to say"].notnull()
responses_cleaned['reason_noschooltalks'] = responses["My school doesn't provide talks like these at all"].notnull()
responses_cleaned['reason_noguidance'] = responses["I don't know who to ask about college applications overseas"].notnull()
responses_cleaned['reason_teacher'] = responses["My teacher told me to sign up for this"].notnull()
responses_cleaned['reason_friend'] = responses["I heard that my friends were signing up, so I decided to sign up too"].notnull()

responses_cleaned['rating'] = responses["Rate your overall experience"]
responses_cleaned['continued_interest'] = responses["I would be keen to attend more of such webinars"]
responses_cleaned['enjoyed_session'] = responses["I enjoyed the session"]
responses_cleaned['learnt_session'] = responses["I learnt a lot from the session"]
responses_cleaned['motivatedby_session'] = responses["I am more motivated to join the same industry as the speakers after this sharing"]
responses_cleaned['mentor_interest'] = responses["I am interested in finding a mentor (for advice on future life)."]
responses_cleaned['mentor_find_difficulty'] = responses["How difficult is it to find a mentor?"]
responses_cleaned['major_info_difficulty'] = responses["How easy is it to find information about your desired major?"]

afi_cols = ["What do you think *could have improved* in this AMA webinar?","What are some things we could have done to make the session better?","Area(s) for Improvement that I would like to suggest"]
afi_dfs = [responses[colname] for colname in afi_cols]
responses_cleaned['improvements'] = reduce(lambda x,y: x.combine_first(y), afi_dfs).astype(str).str.lower()

takeaway_cols = ["What is your greatest takeaway from this session?","What are some actionables that you would set for yourself after this webinar?/  How has your perspective changed?"]
takeaway_dfs = [responses[colname] for colname in takeaway_cols]
responses_cleaned['takeaway'] = reduce(lambda x,y: x.combine_first(y), takeaway_dfs).astype(str).str.lower()

responses_cleaned['most_helpful'] = responses['What did you find most helpful in the webinar?'].astype(str)
responses_cleaned['speaker_feedback'] = responses["Anything that you would like to say to any of our speakers? :)"].astype(str)
responses_cleaned['other_comments'] = responses["Do you have any other feedback for the team? "].astype(str).combine_first(responses["Any other feedback, comments or suggestions:"]).astype(str)
responses_cleaned['app_rating']=responses['The CareerContact app is user-friendly'].astype(str) 
responses_cleaned['other_profession'] = reduce(lambda x,y: x.combine_first(y), other_profession_dfs).astype(str)

The below cell creates the generalize() function which helps in mapping large lists of categories (like school data). Though this process is memory intensive, it will be useful and easy to maintain as the number of schools and the exceptions increase.

In [6]:
def generalize(ser, match_name, default=None, regex=False, case=False):
    """ Search a series for text matches.
    Based on code from https://www.metasnake.com/blog/pydata-assign.html

    ser: pandas series to search
    match_name: tuple containing text to search for and text to use for normalization
    default: If no match, use this to provide a default value, otherwise use the original text
    regex: Boolean to indicate if match_name contains a  regular expression
    case: Case sensitive search

    Returns a pandas series with the matched value

    """
    seen = None
    for match, name in match_name:
        mask = ser.str.contains(match, case=case, regex=regex, na=False)
        if seen is None:
            seen = mask
        else:
            seen |= mask
        ser = ser.where(~mask, name)
    if default:
        ser = ser.where(seen, default)
    else:
        ser = ser.where(seen, ser.values)
    return ser

school_patterns = [('Hwa Chong', 'Hwa Chong Institution'), ('School of the Arts','School of the Arts'),
                   ('Eunoia', 'Eunoia Junior College'),('Jurong Pioneer', 'Jurong Pioneer Junior College'),
                   ('Saint Andrew', "Saint Andrew's Junior College"),('Jurong Pioneer', 'Jurong Pioneer Junior College'),
                   ('CJC', "Catholic Junior College")]
responses_cleaned['school'] = generalize(responses["What's your school?"], school_patterns)
school_col = responses_cleaned.pop('school')
responses_cleaned.insert(2, 'school', school_col)

grade_patterns = [('J1', '11'), ('J2','12'),('J3', '13'),('11', '11'), ('12','12'),('13', '13')]
responses_cleaned['grade'] = generalize(responses["Which grade are you in?"], grade_patterns)
grade_col = responses_cleaned.pop('grade')
responses_cleaned.insert(3, 'grade', grade_col)

In [7]:
responses_cleaned['knowledge'] = reduce(lambda x,y: x.combine_first(y), knowledge_dfs).astype(str) 
responses_cleaned['interest'] = reduce(lambda x,y: x.combine_first(y), interest_dfs).astype(str)  
responses_cleaned['learn'] = reduce(lambda x,y: x.combine_first(y), learn_dfs).astype(str)  
responses_cleaned['helpful'] = responses["Let's get going. How helpful was the advice that Monica and Ashton gave?"].astype(str)

def knowledge_categories(df_string):
    if df_string.startswith('I know little or nothing about'):
        return 1
    elif 'certainty' in df_string:
        return 2
    elif df_string.startswith('I am able to articulate') and '& more' not in df_string and '& beyond' not in df_string:
        return 3
    elif df_string.startswith('I am able to articulate') and ('& more, e.g. challenges and trends' in df_string or '& beyond' in df_string):
        return 4

def interest_categories(df_string):
    if df_string.startswith('I am now more likely'):
        return 1
    elif df_string.startswith('I am now less likely'):
        return 2
    elif df_string.startswith('I am neither more nor less'):
        return 3
    elif 'undecided' in df_string:
        return 4

def learn_categories(df_string):
    if df_string == 'Agree':
        return 1
    elif df_string == 'Somewhat Agree' :
        return 2
    elif df_string == 'Somewhat Disagree':
        return 3
    elif df_string == 'Disagree':
        return 4

def helpful_categories(df_string):
    if df_string.startswith('Really unhelpful'):
        return 1
    elif df_string.startswith('Not so helpful'):
        return 2
    elif df_string.startswith('It was alright'):
        return 3
    elif df_string.startswith('Helpful'):
        return 4
    elif df_string.startswith('Really helpful'):
        return 5

responses_cleaned['knowledge_coded'] = responses_cleaned.knowledge.apply(knowledge_categories)
responses_cleaned['interest_coded'] = responses_cleaned.interest.apply(interest_categories)
responses_cleaned['learn_coded'] = responses_cleaned.learn.apply(learn_categories)
responses_cleaned['helpful_coded'] = responses_cleaned.helpful.apply(helpful_categories)

responses_cleaned['learn_coded']= responses_cleaned['learn_coded'].combine_first(responses_cleaned['helpful_coded'])
responses_cleaned['learn_coded']= responses_cleaned['learn_coded'].combine_first(responses_cleaned['learnt_session'])
responses_cleaned=responses_cleaned.drop(columns=["interest","learn","knowledge","helpful","helpful_coded",'learnt_session'])


Export the dataframe!

In [9]:
pd.to_pickle(responses_cleaned, 'data/postevent_responses_2.pandas')